<a href="https://colab.research.google.com/github/obedotto/rl-value-iteration/blob/main/RL_Value_Iteration.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import warnings ; warnings.filterwarnings('ignore')

import gym, gym_walk
import numpy as np

import random
import warnings

warnings.filterwarnings('ignore', category=DeprecationWarning)
np.set_printoptions(suppress=True)
random.seed(123); np.random.seed(123)


Gym has been unmaintained since 2022 and does not support NumPy 2.0 amongst other critical functionality.
Please upgrade to Gymnasium, the maintained drop-in replacement of Gym, or contact the authors of your software and request that they upgrade.
See the migration guide at https://gymnasium.farama.org/introduction/migration_guide/ for additional information.


In [ ]:
pip install git+https://github.com/mimoralea/gym-walk#egg=gym-walk

  Cloning https://github.com/mimoralea/gym-walk to /tmp/pip-install-57x9l2lx/gym-walk_23b39f0d9bca49f1a8abb3e8687f498d
  Running command git clone --filter=blob:none --quiet https://github.com/mimoralea/gym-walk /tmp/pip-install-57x9l2lx/gym-walk_23b39f0d9bca49f1a8abb3e8687f498d
  Resolved https://github.com/mimoralea/gym-walk to commit b915b94cf2ad16f8833a1ad92ea94e88159279f5
  Preparing metadata (setup.py) ... done
  Created wheel for gym-walk: filename=gym_walk-0.0.2-py3-none-any.whl size=5377 sha256=077443152f0489a0dd8fe89f1b486f397aee59890cd2c07fd70ed44705d10584
  Stored in directory: /tmp/pip-ephem-wheel-cache-i4vuce26/wheels/bf/23/e5/a94be4a90dd18f7ce958c21f192276cb01ef0daaf2bc66583b
Successfully built gym-walk


In [ ]:
def print_policy(pi, P, action_symbols=('<', 'v', '>', '^'), n_cols=4, title='Policy:'):
    print(title)
    arrs = {k:v for k,v in enumerate(action_symbols)}
    for s in range(len(P)):
        a = pi(s)
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), arrs[a].rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
def print_state_value_function(V, P, n_cols=4, prec=3, title='State-value function:'):
    print(title)
    for s in range(len(P)):
        v = V[s]
        print("| ", end="")
        if np.all([done for action in P[s].values() for _, _, _, done in action]):
            print("".rjust(9), end=" ")
        else:
            print(str(s).zfill(2), '{}'.format(np.round(v, prec)).rjust(6), end=" ")
        if (s + 1) % n_cols == 0: print("|")

In [ ]:
def probability_success(env, pi, goal_state, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        while not done and steps < max_steps:
            state, _, done, h = env.step(pi(state))
            steps += 1
        results.append(state == goal_state)
    return np.sum(results)/len(results)

In [ ]:
def mean_return(env, pi, n_episodes=100, max_steps=200):
    random.seed(123); np.random.seed(123) ; env.seed(123)
    results = []
    for _ in range(n_episodes):
        state, done, steps = env.reset(), False, 0
        results.append(0.0)
        while not done and steps < max_steps:
            state, reward, done, _ = env.step(pi(state))
            results[-1] += reward
            steps += 1
    return np.mean(results)

# Creating the Frozen Lake environment

In [ ]:
envdesc=['HGFF','FFHF','FFFF','HHFS']
env = gym.make('FrozenLake-v1',desc=envdesc)
init_state = env.reset()
goal_state = 1
P = env.env.P

In [ ]:
P

{0: {0: [(1.0, 0, 0, True)],
  1: [(1.0, 0, 0, True)],
  2: [(1.0, 0, 0, True)],
  3: [(1.0, 0, 0, True)]},
 1: {0: [(1.0, 1, 0, True)],
  1: [(1.0, 1, 0, True)],
  2: [(1.0, 1, 0, True)],
  3: [(1.0, 1, 0, True)]},
 2: {0: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 1.0, True),
   (0.3333333333333333, 6, 0.0, True)],
  1: [(0.3333333333333333, 1, 1.0, True),
   (0.3333333333333333, 6, 0.0, True),
   (0.3333333333333333, 3, 0.0, False)],
  2: [(0.3333333333333333, 6, 0.0, True),
   (0.3333333333333333, 3, 0.0, False),
   (0.3333333333333333, 2, 0.0, False)],
  3: [(0.3333333333333333, 3, 0.0, False),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 1, 1.0, True)]},
 3: {0: [(0.3333333333333333, 3, 0.0, False),
   (0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 7, 0.0, False)],
  1: [(0.3333333333333333, 2, 0.0, False),
   (0.3333333333333333, 7, 0.0, False),
   (0.3333333333333333, 3, 0.0, False)],
  2: [(0.3333333333333333, 7, 0.0, Fals

# Value Iteration Algorithm

In [ ]:
def value_iteration(P, gamma=1.0, theta=1e-10):
    V = np.zeros(len(P), dtype=np.float64)
    while True:
      Q=np.zeros((len(P),len(P[0])),dtype=np.float64)
      for s in range(len(P)):
        for a in range(len(P[s])):
          for prob, next_state, reward, done in P[s][a]:
            Q[s][a]+=prob*(reward+gamma*V[next_state]*(not done))
      if np.max(np.abs(V-np.max(Q,axis=1)))<theta:
        break
      V=np.max(Q,axis=1)
    pi=lambda s: {s:a for s,a in enumerate(np.argmax(Q,axis=1))}[s]
    return V, pi

In [ ]:
# Finding the optimal policy
V_best_v, pi_best_v = value_iteration(P, gamma=0.99)


In [ ]:
# Printing the policy
print("Name: SANDHIYA R               Register Number:212223240146")
print('Optimal policy and state-value function (VI):')
print_policy(pi_best_v, P)

Name: SANDHIYA R               Register Number:212223240146
Optimal policy and state-value function (VI):
Policy:
|           |           | 02      ^ | 03      ^ |
| 04      v | 05      < |           | 07      > |
| 08      ^ | 09      ^ | 10      v | 11      ^ |
|           |           | 14      > | 15      > |


In [ ]:
# printing the success rate and the mean return
print('Reaches goal {:.2f}%. Obtains an average undiscounted return of {:.4f}.'.format(
    probability_success(env, pi_best_v, goal_state=goal_state)*100,
    mean_return(env, pi_best_v)))

Reaches goal 100.00%. Obtains an average undiscounted return of 1.0000.


In [ ]:
# printing the state value function
print_state_value_function(V_best_v, P, prec=4)

State-value function:
|           |           | 02 0.9532 | 03 0.9251 |
| 04 0.8393 | 05 0.8833 |           | 07 0.8586 |
| 08 0.8208 | 09 0.8271 | 10 0.8024 | 11 0.8181 |
|           |           | 14 0.7863 | 15 0.7941 |
